In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
import regex as re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet , stopwords
import string
from sklearn.metrics.pairwise import sigmoid_kernel

In [4]:
df = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
df

FileNotFoundError: [Errno 2] File ../input/netflix-shows/netflix_titles.csv does not exist: '../input/netflix-shows/netflix_titles.csv'

In [9]:
df.isnull().sum()

NameError: name 'df' is not defined

In [5]:
df.fillna('',inplace=True)

In [6]:
stopword = stopwords.words('english')

In [7]:
def get_wordnet_pos(word):

    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN) 

In [8]:
lemmatizer = WordNetLemmatizer()
def clean(text):
    text = text.lower()
    text = re.sub(r'[^a-z1-9]',' ',text)
    text = " ".join([lemmatizer.lemmatize(x,get_wordnet_pos(x)) for x in text.split() if x not in stopword ])
    return text

In [9]:
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...


In [10]:
df.shape

(7787, 12)

In [11]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [12]:
texts = []
for i in range(df.shape[0]):
    text = df['cast'][i] + " " + df['rating'][i] + " " + df['listed_in'][i] + " " + df['description'][i]
    line = []
    for i in text :
        if i in string.punctuation:
            if i == "%":
                line.append(i)
            else:
                pass
        else:
            line.append(i)
            
    texts.append("".join(line))
   
    

In [13]:
texts[0]

'João Miguel Bianca Comparato Michel Gomes Rodolfo Valente Vaneza Oliveira Rafael Lozano Viviane Porto Mel Fronckowiak Sergio Mamberti Zezé Motta Celso Frateschi TVMA International TV Shows TV Dramas TV SciFi  Fantasy In a future where the elite inhabit an island paradise far from the crowded slums you get one chance to join the 3% saved from squalor'

In [14]:
all_df = pd.DataFrame(texts,columns= ['clean'])
all_df

,clean
0,João Miguel Bianca Comparato Michel Gomes Rodo...
1,Demián Bichir Héctor Bonilla Oscar Serrano Aza...
2,Tedd Chan Stella Chung Henley Hii Lawrence Koh...
3,Elijah Wood John C Reilly Jennifer Connelly Ch...
4,Jim Sturgess Kevin Spacey Kate Bosworth Aaron ...
...,...
7782,Imad Creidi Antoinette Turk Elias Gergi Carmen...
7783,Vicky Kaushal SarahJane Dias Raaghav Chanana M...
7784,Nasty C TVMA Documentaries International Movie...
7785,Adriano Zumbo Rachel Khoo TVPG International T...


In [15]:
df = pd.concat((df,all_df),axis=1)
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,clean
0,s1,TV Show,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,João Miguel Bianca Comparato Michel Gomes Rodo...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Demián Bichir Héctor Bonilla Oscar Serrano Aza...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",Tedd Chan Stella Chung Henley Hii Lawrence Koh...
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",Elijah Wood John C Reilly Jennifer Connelly Ch...
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,Jim Sturgess Kevin Spacey Kate Bosworth Aaron ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...,Imad Creidi Antoinette Turk Elias Gergi Carmen...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...,Vicky Kaushal SarahJane Dias Raaghav Chanana M...
7784,s7785,Movie,Zulu Man in Japan,,Nasty C,,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast...",Nasty C TVMA Documentaries International Movie...
7785,s7786,TV Show,Zumbo's Just Desserts,,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...,Adriano Zumbo Rachel Khoo TVPG International T...


In [16]:
df['description_cleaned'] = df['clean'].apply(lambda x : clean(x) )

In [17]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,clean,description_cleaned
0,s1,TV Show,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,João Miguel Bianca Comparato Michel Gomes Rodo...,jo miguel bianca comparato michel gomes rodolf...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Demián Bichir Héctor Bonilla Oscar Serrano Aza...,demi n bichir h ctor bonilla oscar serrano aza...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",Tedd Chan Stella Chung Henley Hii Lawrence Koh...,tedd chan stella chung henley hii lawrence koh...
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",Elijah Wood John C Reilly Jennifer Connelly Ch...,elijah wood john c reilly jennifer connelly ch...
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,Jim Sturgess Kevin Spacey Kate Bosworth Aaron ...,jim sturgess kevin spacey kate bosworth aaron ...


In [18]:
corpus = []
for i in df['description_cleaned']:
    corpus.append(i)

In [19]:
tfidf = TfidfVectorizer(min_df=3,max_features=None,
                     strip_accents = 'unicode',analyzer='word',
                     ngram_range=(1,3))
ncorpus = tfidf.fit_transform(corpus)

In [5]:
ncorpus

NameError: name 'ncorpus' is not defined

In [20]:
tfidf.vocabulary_

{'jo': 10721,
 'miguel': 14365,
 'bianca': 2411,
 'comparato': 4330,
 'michel': 14315,
 'rodolfo': 18731,
 'oliveira': 16231,
 'rafael': 17768,
 'lozano': 13010,
 'porto': 17348,
 'mel': 14113,
 'sergio': 19794,
 'celso': 3466,
 'tvma': 23004,
 'international': 9908,
 'tv': 22768,
 'show': 20209,
 'drama': 5975,
 'scifi': 19547,
 'fantasy': 7159,
 'future': 7908,
 'elite': 6430,
 'inhabit': 9831,
 'island': 10144,
 'paradise': 16622,
 'far': 7174,
 'crowd': 4694,
 'slum': 20538,
 'get': 8145,
 'one': 16287,
 'chance': 3546,
 'join': 10875,
 'save': 19432,
 'tvma international': 23049,
 'international tv': 10021,
 'tv show': 22870,
 'show tv': 20268,
 'tv drama': 22801,
 'drama tv': 6042,
 'tv scifi': 22867,
 'scifi fantasy': 19548,
 'get one': 8167,
 'one chance': 16289,
 'tvma international tv': 23051,
 'international tv show': 10022,
 'tv show tv': 22899,
 'show tv drama': 20271,
 'tv drama tv': 22829,
 'drama tv scifi': 6045,
 'tv scifi fantasy': 22868,
 'demi': 5335,
 'bichir': 241

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
similarity = cosine_similarity(ncorpus)

In [31]:
sorted(list(enumerate(similarity[0])) , key = lambda x : x[1],reverse = True)

[(0, 1.0000000000000002),
 (984, 0.2465066370223824),
 (4611, 0.2360052725244666),
 (931, 0.2177531709286489),
 (1736, 0.19604589721251423),
 (5791, 0.1945537718281115),
 (3416, 0.1897083013375314),
 (6961, 0.18110635365563016),
 (640, 0.17868866607209727),
 (5110, 0.17864006939955548),
 (1064, 0.17854225644946178),
 (2593, 0.17688108554266074),
 (6215, 0.17654863345002814),
 (7323, 0.17581262708149376),
 (1555, 0.17427341871986946),
 (575, 0.17202467747996),
 (3568, 0.17084875742819644),
 (3289, 0.17048579045425194),
 (6476, 0.16474492477953118),
 (680, 0.16230145913744792),
 (3219, 0.1622623222962733),
 (1415, 0.15726950455278152),
 (6849, 0.15654374529817344),
 (3733, 0.15620724457728405),
 (3184, 0.15619643129599786),
 (4562, 0.15361550484012612),
 (3789, 0.15348799138263092),
 (4631, 0.15084419099507604),
 (2491, 0.15016622949861178),
 (5943, 0.1498337128368719),
 (5286, 0.14937320635799917),
 (2955, 0.14869809198214282),
 (6907, 0.14781894162677492),
 (1465, 0.14708382875390946),

In [21]:
sig = sigmoid_kernel(ncorpus,ncorpus)

In [22]:
indices = pd.Series(df['title'].index, index = df['title']).drop_duplicates()
indices

title
3%                                            0
7:19                                          1
23:59                                         2
9                                             3
21                                            4
                                           ... 
Zozo                                       7782
Zubaan                                     7783
Zulu Man in Japan                          7784
Zumbo's Just Desserts                      7785
ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS    7786
Length: 7787, dtype: int64

In [23]:

sorted(list(enumerate(sig[0])) , key = lambda x : x[1],reverse = True)

[(0, 0.7616110524141492),
 (984, 0.7615983211410644),
 (4611, 0.7615981437023825),
 (931, 0.7615978353013457),
 (1736, 0.7615974685187106),
 (5791, 0.7615974433066054),
 (3416, 0.7615973614337701),
 (6961, 0.761597216088517),
 (640, 0.761597175237358),
 (5110, 0.7615971744162302),
 (1064, 0.7615971727635051),
 (2593, 0.7615971446950458),
 (6215, 0.7615971390776715),
 (7323, 0.7615971266415237),
 (1555, 0.7615971006338358),
 (575, 0.761597062637312),
 (3568, 0.7615970427680276),
 (3289, 0.7615970366350474),
 (6476, 0.7615969396327736),
 (680, 0.7615968983460069),
 (3219, 0.761596897684719),
 (1415, 0.7615968133220229),
 (6849, 0.761596801059004),
 (3733, 0.7615967953732138),
 (3184, 0.7615967951905038),
 (4562, 0.7615967515810651),
 (3789, 0.7615967494264936),
 (4631, 0.7615967047546781),
 (2491, 0.7615966932992846),
 (5943, 0.76159668768081),
 (5286, 0.7615966798997145),
 (2955, 0.7615966684924271),
 (6907, 0.7615966536375796),
 (1465, 0.7615966412165072),
 (7500, 0.7615966281201902),


In [32]:
def rec_cosine(title,sig=similarity):
    idx = indices[title]
    sig_score = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_score, key = lambda x : x[1],reverse = True)
    sig_scores = sig_scores[1:11]
    movies_indices = [i[0] for i in sig_scores]
    return df.iloc[movies_indices]

In [24]:
def rec(title,sig=sig):
    idx = indices[title]
    sig_score = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_score, key = lambda x : x[1],reverse = True)
    sig_scores = sig_scores[1:11]
    movies_indices = [i[0] for i in sig_scores]
    return df.iloc[movies_indices]

In [25]:
rec('Zubaan')


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,clean,description_cleaned
6469,s6470,Movie,The Human Factor: The Untold Story of the Bomb...,Rudradeep Bhattacharjee,,India,"April 1, 2017",2013,TV-PG,53 min,"Documentaries, International Movies, Music & M...",A family of Parsi musicians collectively works...,TVPG Documentaries International Movies Music...,tvpg documentary international movie music mus...
5403,s5404,Movie,Satrangi Re,Aditya Sarpotdar,"Amruta Khanvilkar, Vidyadhar Joshi, Subodh Bhave",India,"March 15, 2018",2012,TV-14,150 min,"Dramas, International Movies, Music & Musicals",Friends in their last year of college contempl...,Amruta Khanvilkar Vidyadhar Joshi Subodh Bhave...,amruta khanvilkar vidyadhar joshi subodh bhave...
594,s595,Movie,Asees,Rana Ranbir,"Rana Ranbir, Neha Pawar, Kuljinder Sidhu, Rupi...",India,"November 1, 2018",2018,TV-14,123 min,"Dramas, International Movies, Music & Musicals",When a dispute breaks out between five sibling...,Rana Ranbir Neha Pawar Kuljinder Sidhu Rupinde...,rana ranbir neha pawar kuljinder sidhu rupinde...
987,s988,Movie,Blood Money,Vishal Mahadkar,"Manish Chaudhary, Kunal Kemmu, Amrita Puri, Sh...",India,"July 5, 2020",2012,TV-14,110 min,"Dramas, International Movies, Romantic Movies",Ambitious Kunal takes a job in the diamond bus...,Manish Chaudhary Kunal Kemmu Amrita Puri Sheka...,manish chaudhary kunal kemmu amrita puri sheka...
5377,s5378,Movie,Sanju,Rajkumar Hirani,"Ranbir Kapoor, Vicky Kaushal, Paresh Rawal, So...",India,"October 15, 2018",2018,TV-MA,159 min,"Dramas, International Movies",From the depths of addiction to the heights of...,Ranbir Kapoor Vicky Kaushal Paresh Rawal Sonam...,ranbir kapoor vicky kaushal paresh rawal sonam...
910,s911,Movie,Bigflo & Oli: Hip Hop Frenzy,"Bigflo & Oli, Jérémie Levypon",Bigflo & Oli,France,"October 8, 2020",2020,TV-MA,100 min,"Documentaries, International Movies, Music & M...",Go backstage with French rap duo Bigflo & Oli ...,Bigflo Oli TVMA Documentaries International M...,bigflo oli tvma documentary international movi...
5100,s5101,Movie,Rangreza,Amir Mohiuddin,"Urwa Hocane, Bilal Ashraf, Gohar Rasheed, Sale...",Pakistan,"August 1, 2018",2017,TV-14,126 min,"Dramas, International Movies, Music & Musicals",A high-flying rock star and son of a politicia...,Urwa Hocane Bilal Ashraf Gohar Rasheed Saleem ...,urwa hocane bilal ashraf gohar rasheed saleem ...
7498,s7499,Movie,We Are One,Stéphane de Freitas,,France,"July 14, 2020",2020,TV-MA,86 min,"Documentaries, International Movies, Music & M...",Activists around the world fight injustice and...,TVMA Documentaries International Movies Music...,tvma documentary international movie music mus...
7472,s7473,Movie,Wandering Stars,"Syrine Boulanouar, Nekfeu",Nekfeu,France,"December 1, 2019",2019,TV-MA,95 min,"Documentaries, International Movies, Music & M...",Navigating creative pressures and personal ups...,Nekfeu TVMA Documentaries International Movies...,nekfeu tvma documentary international movie mu...
7051,s7052,Movie,This Was Tomorrow,Wim Bonte,,Belgium,"December 15, 2015",2015,NR,77 min,"Documentaries, International Movies, Music & M...",The Tomorrowland music festival marks its 10th...,NR Documentaries International Movies Music ...,nr documentary international movie music music...


In [33]:
rec_cosine('Zubaan')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,clean,description_cleaned
6469,s6470,Movie,The Human Factor: The Untold Story of the Bomb...,Rudradeep Bhattacharjee,,India,"April 1, 2017",2013,TV-PG,53 min,"Documentaries, International Movies, Music & M...",A family of Parsi musicians collectively works...,TVPG Documentaries International Movies Music...,tvpg documentary international movie music mus...
5403,s5404,Movie,Satrangi Re,Aditya Sarpotdar,"Amruta Khanvilkar, Vidyadhar Joshi, Subodh Bhave",India,"March 15, 2018",2012,TV-14,150 min,"Dramas, International Movies, Music & Musicals",Friends in their last year of college contempl...,Amruta Khanvilkar Vidyadhar Joshi Subodh Bhave...,amruta khanvilkar vidyadhar joshi subodh bhave...
594,s595,Movie,Asees,Rana Ranbir,"Rana Ranbir, Neha Pawar, Kuljinder Sidhu, Rupi...",India,"November 1, 2018",2018,TV-14,123 min,"Dramas, International Movies, Music & Musicals",When a dispute breaks out between five sibling...,Rana Ranbir Neha Pawar Kuljinder Sidhu Rupinde...,rana ranbir neha pawar kuljinder sidhu rupinde...
987,s988,Movie,Blood Money,Vishal Mahadkar,"Manish Chaudhary, Kunal Kemmu, Amrita Puri, Sh...",India,"July 5, 2020",2012,TV-14,110 min,"Dramas, International Movies, Romantic Movies",Ambitious Kunal takes a job in the diamond bus...,Manish Chaudhary Kunal Kemmu Amrita Puri Sheka...,manish chaudhary kunal kemmu amrita puri sheka...
5377,s5378,Movie,Sanju,Rajkumar Hirani,"Ranbir Kapoor, Vicky Kaushal, Paresh Rawal, So...",India,"October 15, 2018",2018,TV-MA,159 min,"Dramas, International Movies",From the depths of addiction to the heights of...,Ranbir Kapoor Vicky Kaushal Paresh Rawal Sonam...,ranbir kapoor vicky kaushal paresh rawal sonam...
910,s911,Movie,Bigflo & Oli: Hip Hop Frenzy,"Bigflo & Oli, Jérémie Levypon",Bigflo & Oli,France,"October 8, 2020",2020,TV-MA,100 min,"Documentaries, International Movies, Music & M...",Go backstage with French rap duo Bigflo & Oli ...,Bigflo Oli TVMA Documentaries International M...,bigflo oli tvma documentary international movi...
5100,s5101,Movie,Rangreza,Amir Mohiuddin,"Urwa Hocane, Bilal Ashraf, Gohar Rasheed, Sale...",Pakistan,"August 1, 2018",2017,TV-14,126 min,"Dramas, International Movies, Music & Musicals",A high-flying rock star and son of a politicia...,Urwa Hocane Bilal Ashraf Gohar Rasheed Saleem ...,urwa hocane bilal ashraf gohar rasheed saleem ...
7498,s7499,Movie,We Are One,Stéphane de Freitas,,France,"July 14, 2020",2020,TV-MA,86 min,"Documentaries, International Movies, Music & M...",Activists around the world fight injustice and...,TVMA Documentaries International Movies Music...,tvma documentary international movie music mus...
7472,s7473,Movie,Wandering Stars,"Syrine Boulanouar, Nekfeu",Nekfeu,France,"December 1, 2019",2019,TV-MA,95 min,"Documentaries, International Movies, Music & M...",Navigating creative pressures and personal ups...,Nekfeu TVMA Documentaries International Movies...,nekfeu tvma documentary international movie mu...
7051,s7052,Movie,This Was Tomorrow,Wim Bonte,,Belgium,"December 15, 2015",2015,NR,77 min,"Documentaries, International Movies, Music & M...",The Tomorrowland music festival marks its 10th...,NR Documentaries International Movies Music ...,nr documentary international movie music music...


In [4]:
a_file = open("test.txt", "w")
for row in ncorpus.toarray():
    np.savetxt(a_file, row)

NameError: name 'ncorpus' is not defined

In [ ]:
d = np.loadtxt("test.txt")

In [ ]:
df.to_csv("dataframe.csv")